# 1. 텐서플로

- 강력한 수치 계산용 라이브러리
- 구글 브레인 팀에서 개발
- 구글 클라우드 스피치, 구글 포토, 구글 검색 대규모 서비스에 사용됨
- 2015년 11월 오픈 소스 공개
- GPU 지원
- 분산 컴퓨팅 지원
- Just-In-Time 컴파일러 포함 - 소도를 높이고 메모리 사용량을 즐이기 위해 계산을 최적화함
- 다른 환경에서 훈련과 실행이 가능하다
- 자동 미분 기능과 고성능 옵티마이저 RMSProp, Nadam 제공
- TensorFlow.js 자바스크립트 구현 있음, 브라우저에서 직접 모델 실행 가능한 것

저수준 텐서플로 연산 
- C++ 코드로 구현
- 여러 구현(kernel)이 존재 
- kernel은 CPU, GPU, TPU 같은 특정 장치에 맞춰 만들어짐

[TensorFlow GitHub](https://github.com/jtoy/awesome-tensorflow) 에서 많은 신경망 구조 다운로드 가능

TFX : TensorFlow 제품화를 위한 라이브러리 모음

[TensorFlow 버그 알리기, 새로운 기능 요청하기](https://github.com/tensorflow/tensorflow)    
[TensorFlow 커뮤니티](https://homl.info/41)

# 2. Using the TensorFlow LIKE NUMPY

##### tensor
- 한 연산에서 다른 연산으로 ... ~ tensorflow
- numpy - ndarray와 매우 비슷함
- 다차원 배열, 스칼라 값 가능    
텐서 생성과 조작

## 2.1 텐서와 연산
tf.constant()

1. 행렬

In [2]:
import tensorflow as tf

tf.constant([[1., 2., 3.], [4., 5., 6.]]) # matrix

2023-03-09 20:51:44.572241: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-09 20:52:39.707103: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

2. 스칼라

In [3]:
tf.constant(42) # scalar

<tf.Tensor: shape=(), dtype=int32, numpy=42>

3. 크기와 데이터 타입

In [4]:
t = tf.constant([[1., 2., 3.], [4., 5., 6.]])
t, t.shape, t.dtype

(<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
 array([[1., 2., 3.],
        [4., 5., 6.]], dtype=float32)>,
 TensorShape([2, 3]),
 tf.float32)

## 2.5 데이터 구조
- 희소 텐서
- 텐서 배열 : 텐서의 리스트. 리스트에 포함된 모든 텐서 크기와 데이터 타입은 동일해야함
- 래그드 텐서 : 리스트의 리스트. 리스트의 길이는 다를 수 있음. tf.ragged package
- 문자열 텐서 : tf.string . 바이트 문자열을 나타냄(유니코드 아님!) 자동으로 UTF-8로 인코딩
- 집합 : 텐서 혹은 희소텐서로 표현. tf.sets package
- 큐 : tf.queue 

타입 변환 : tf.cast()

```python
tf.cast(t2, tf.float32)
```

## 2.4 변수
텐서의 내용은 수정 불가 = 역전파시 신경망의 가중치를 변경할 수 없음
-> tf.Variable 사용

In [5]:
v = tf.Variable([[1., 2., 3.], [4., 5., 6.]])
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

- tf.Variable은 tf.Tensor와 비슷하게 작동함
- assign() 이나 scatter_update(), scatter_nd_update() 로 개별 원소를 수정 가능

In [6]:
v.assign(2 * v)
print(v)

v.scatter_nd_update(indices=[[0, 0], [1, 2]],
                    updates=[100., 200.])
print(v)

sparse_delta = tf.IndexedSlices(values=[[1., 2., 3.], [4., 5., 6.]],
                                indices=[1, 0])
v.scatter_update(sparse_delta)
print(v)

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>
<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[100.,   4.,   6.],
       [  8.,  10., 200.]], dtype=float32)>
<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[4., 5., 6.],
       [1., 2., 3.]], dtype=float32)>


## 2.2 Tensor and Numpy

넘파이 배열에 텐서플로 연산 가능    
텐서에 넘파이 연산 가능

## 2.3 타입 변환
